## Vector Search: From Zero To Hero
这一系列代码教程是为了让你在学习后能够对向量搜索有一定的认识和了解，通过该教程你可以
- 对向量搜索的算法有一个清晰的认识
- 自己动手实现一个简单的向量搜索库
- 对不同的向量搜索算法有一个直观的感受

该教程使用Python实现，如果你没有学过Python也不用紧张，Python十分简单易懂，并且该教程更专注于算法实现。

该教程使用sift1M作为数据集。

运行下面的code block，然后开始向量搜索之旅。

In [ ]:
print('Welcome to Vector Search! Hope you will become Vector Search Hero!')

### Prepare Data
如果你已经下载了数据，可以直接跳过这个cell

In [1]:
import shutil
import urllib.request as request
import tarfile
from contextlib import closing

# download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/siftsmall.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)


# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()


### The K-Nearest Neighbors (kNN)
KNN 是向量搜索中最简单的一个算法，首先想象一个二维平面中有许多的点，当你想要得到距离某一个点最近的K个点时，最朴素的算法就是遍历所有的点，挨个计算距离
并始终保存K个最近的点，当遍历结束，也就得到了最近的K个点。

向量搜索中大部分的向量都是高维向量，但是思想是相同的，因此KNN算法可以很轻易的从二维空间延展到多维空间。

KNN的缺点
1. 计算量大
2. 空间复杂度高

优点就是简单易于实现，因此这一份教程以KNN作为开篇，让你可以更容易熟悉这个教程的流程。

<img src="./resources/knn.png" title="From Weaviate" width="50%" height="50%">

Picture From Weaviate

#### DISTANCE
既然要算出最近的距离，我们就需要有一个距离函数。

KNN算法中我们使用欧式距离，公式如下
$$y=\sqrt{\sum_{i=1}^{n}{\left| x_{i}-y_{i} \right|^{2}}}$$

如果你对向量搜索中使用的距离公式感兴趣,有一篇很好的博客可供参考

[distance in vector search](https://weaviate.io/blog/distance-metrics-in-vector-search)

##### **TASK1**:
你需要在KNN类中的distance函数实现这一公式， **u**,**v**为numpy的数组类型，你可以假设他们都是一维数组，并且size完全相等。
##### **HINT**:
1. 如果你并不熟悉numpy也无需慌张,这里有一份很好的[tutorial](https://numpy.org/doc/stable/user/absolute_beginners.html)，可以帮助你快速的熟悉numpy。
2. 无需着急，第一个task十分简单，他的主要目的是让你熟悉numpy,你大可以放松心情。
3. python有许多有用的内置函数，你可以在[link](https://docs.python.org/3/library/functions.html)中找到。也许sum，zip可以帮助到你。
4. 如果你对python不熟悉，这里有一份[tutorial](https://docs.python.org/3/tutorial/index.html),

在你实现距离函数后，你可以尝试运行DISTANCE FUNCTION TEST的CELL，如果不出意外的话，你可以看到**Distance Pass**

##### **TASK2**
在完成了距离计算后，你需要实现KNN类中的topK函数，你需要计算**query**与**data_set**中每一个向量的距离，并从中选出距离最短的K个向量并返回
你不需要返回原始的向量，你只需要返回对应的id即可，即如果
````
self.data_set[i]
self.data_set[j]
self.data_set[k]
```` 
为距离最近的三个向量
那么你只需要返回 **[i, j, k]** 即可.
##### **HINT**:
1. 也许python中的[heapq](https://docs.python.org/3/library/heapq.html)对你有所帮助
2. 我们使用的数据集规模为 **(200000,128)**, 因此测试可能需要几分钟.

不出意外的话，你可以看到 **KNN pass**

从测试中，你应该也体会到了，KNN的耗时让人难以忍受.因此实际生产环境中，我们不会使用KNN来进行向量搜索，而是使用ANN(Approximate Nearest Neighbor),
这实际上是一种以精准度换时间的做法，而这也就来到了向量搜索最有趣的部分。GLHF!

In [14]:
import numpy as np


def read_fvecs(file: str) -> np.ndarray:
    fv = np.fromfile(file, dtype=np.float32)
    if fv.size == 0:
        return np.zeros((0, 0))
    dim = fv.view(np.int32)[0]
    return fv.reshape(-1, dim + 1)[:, 1:].copy().view('float32')


def read_ivecs(file: str) -> np.ndarray:
    ground_truth = np.fromfile(file, dtype=np.int32)
    if ground_truth.size == 0:
        return ground_truth.zeros((0, 0))
    d = ground_truth[0]
    return ground_truth.reshape(-1, d+1)[:, 1:].copy().view('int32')


class KNN:

    # Calculate the euclidean distance between V and U
    def distance(self, v: np.ndarray, u: np.ndarray) -> float:
        # your code here
        pass

    # Get the top K nearest point of query in self.data
    # just return point's id ig. it's index of self.data_set
    def topK(self, query: np.ndarray, k: int) -> np.ndarray:
        # your code here
        pass

    def __init__(self) -> None:
        self.data_set = read_fvecs("./siftsmall/siftsmall_base.fvecs")


In [ ]:
# DISTANCE FUNCTION TEST
knn = KNN()
assert knn.distance(np.array([1, 2]), np.array(
    [1, 3])) == 1.0, "FAIL, EXPECT 1.0"
assert knn.distance(np.array([1, 2, 3]), np.array(
    [1, 1, 1])) == 2.23606797749979, "FAIL, EXPECT 2.23"
assert knn.distance(np.array([1, 2]), np.array(
    [-1, 2])) == 2.0, "FAIL, EXPECT 2.0"
assert knn.distance(np.array([]), np.array([])) == 0, "FAIL, EXPECT 0"
assert knn.distance(np.array([1.0]), np.array([1])) == 0, "FAIL, EXPECT 0"
print('\033[92m' + "Distance Pass" + '\033[0m')


In [ ]:
import random
# KNN TEST
knn = KNN()
K = 3
ground_truth = read_ivecs("./siftsmall/siftsmall_groundtruth.ivecs")
querys = read_fvecs("./siftsmall/siftsmall_query.fvecs")

for time in range(0, 3):
    query_idx = random.randint(0, 20)
    expected = ground_truth[query_idx][ground_truth[query_idx]]
    actual = knn.topK(querys[query_idx], K)
    assert np.array_equal(expected[:K], actual), "FAIL"
print('\033[92m' + "KNN Pass" + '\033[0m')
